In [197]:
intrastate_coup_concat.columns

Index(['cowcode', 'event_type1', 'event_type2', 'Year', 'Month', 'StateAbb',
       'StateNme'],
      dtype='object')

# Import Stuff

In [198]:
import pandas as pd
import altair as alt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# CSVs

In [199]:
coup = pd.read_csv('https://databank.illinois.edu/datafiles/40o3s/download')
intrastate = pd.read_csv('https://correlatesofwar.org/wp-content/uploads/Intra-StateWarData_v4.1.csv', encoding = 'latin-1', thousands = ',')
cow = pd.read_csv('https://correlatesofwar.org/wp-content/uploads/COW-country-codes.csv')

# Preparing the data

In [200]:
trimmed_coup = pd.DataFrame({'cowcode': coup['cowcode'], 'event_type1': 'Coup', 'event_type2': coup['event_type'], 'Year': coup['year'], 'Month': coup['month']})

In [201]:
intrastate = intrastate.replace(-8, np.nan)
intrastate['CcodeA'] = intrastate['CcodeA'].where(intrastate['CcodeA'].notna(), intrastate['CcodeB'])
intrastate = intrastate.dropna(axis=0, how='all')
trimmed_intrastate = pd.DataFrame({'cowcode': intrastate['CcodeA'], 'event_type1': 'Intra-state war', 'event_type2': intrastate['WarType'], 'Year': intrastate['StartYear1'], 'Month': intrastate['StartMonth1']})

In [202]:
intrastate_coup_concat = pd.concat([trimmed_coup, trimmed_intrastate])
intrastate_coup_concat

,cowcode,event_type1,event_type2,Year,Month
0,439.0,Coup,coup,2014,10
1,516.0,Coup,attempted,2015,5
2,483.0,Coup,attempted,2006,4
3,651.0,Coup,coup,2011,2
4,651.0,Coup,coup,2013,7
...,...,...,...,...,...
437,520.0,Intra-state war,4,2006,3
438,530.0,Intra-state war,4,2006,10
439,531.0,Intra-state war,4,2006,10
440,780.0,Intra-state war,5,2006,10


In [203]:
intrastate_coup_concat['Year'] = pd.to_datetime(intrastate_coup_concat['Year'], format='%Y').dt.year
intrastate_coup_concat = intrastate_coup_concat.where(intrastate_coup_concat['Year'] > 1944, np.nan)
intrastate_coup_concat = intrastate_coup_concat.where(intrastate_coup_concat['Year'] < 2007 , np.nan)
intrastate_coup_concat = intrastate_coup_concat.dropna(axis=0, how='all')

In [204]:
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace('attempted', 'Attempted coup')
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace('conspiracy', 'Conspiracy')
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace('coup', 'Successful coup')
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace(4, 'Civil war for central control')
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace(5, 'Civil war over local issues')
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace(6, 'Regional internal war')
intrastate_coup_concat['event_type2'] = intrastate_coup_concat['event_type2'].replace(7, 'Intercommunal war')
intrastate_coup_concat

,cowcode,event_type1,event_type2,Year,Month
2,483.0,Coup,Attempted coup,2006.0,4.0
5,950.0,Coup,Successful coup,2006.0,12.0
22,800.0,Coup,Successful coup,2006.0,9.0
53,840.0,Coup,Conspiracy,2006.0,2.0
54,552.0,Coup,Conspiracy,2006.0,3.0
...,...,...,...,...,...
435,483.0,Intra-state war,Civil war for central control,2005.0,12.0
437,520.0,Intra-state war,Civil war for central control,2006.0,3.0
438,530.0,Intra-state war,Civil war for central control,2006.0,10.0
439,531.0,Intra-state war,Civil war for central control,2006.0,10.0


In [205]:
cow = cow.rename({'CCode': 'cowcode'}, axis='columns')
intrastate_coup_concat = pd.merge(intrastate_coup_concat, cow, how='left', on='cowcode')

# Viz 1

In [206]:
interval = alt.selection_interval(encodings=['x'])

In [207]:
line_plot = alt.Chart(intrastate_coup_concat).mark_line().encode(alt.X('Year:O'), alt.Y('count():Q', title='# of events'), alt.Color('event_type2:N', title='Event Type', legend=alt.Legend(orient='top')))

In [208]:
line_plot = line_plot.add_selection(interval)

In [209]:
bar_plot = alt.Chart(intrastate_coup_concat).mark_bar().encode(alt.X('StateNme:N', title= 'Country'), alt.Y('count():Q', title='# of events'), alt.Color('event_type2:N', title='Event Type', legend=alt.Legend(orient='bottom')), alt.Row('event_type1:N', title='Coup / Intra-state War')).transform_filter(interval)

In [210]:
dash = alt.vconcat(line_plot, bar_plot)
dash

alt.VConcatChart(...)

# Preparing the Data

In [211]:
coup.rename({'noharm': 'Unharmed', 'injured': 'Injured', 'killed': 'Killed', 'harrest': 'House arrest', 'jailed': 'Jailed', 'tried': 'Tried in court', 'fled':'Fled', 'exile': 'Exiled'}, axis=1, inplace=True)

In [212]:
coup['Killed'].replace(0, np.nan, inplace=True)
coup['Unharmed'].replace(0, np.nan, inplace=True)
coup['Injured'].replace(0, np.nan, inplace=True)
coup['House arrest'].replace(0, np.nan, inplace=True)
coup['Jailed'].replace(0, np.nan, inplace=True)
coup['Tried in court'].replace(0, np.nan, inplace=True)
coup['Fled'].replace(0, np.nan, inplace=True)
coup['Exiled'].replace(0, np.nan, inplace=True)

In [213]:
intrastate['SideADeaths'].replace(-9, np.nan, inplace=True)
intrastate['SideADeaths'].replace(-8, np.nan, inplace=True)
intrastate['SideBDeaths'].replace(-8, np.nan, inplace=True)
intrastate['SideBDeaths'].replace(-9, np.nan, inplace=True)

In [214]:
war_death_counts = intrastate.groupby('StartYear1')[['SideADeaths', 'SideBDeaths']].sum()

In [215]:
war_death_counts['Total Deaths'] = war_death_counts['SideADeaths'].astype('int64') + war_death_counts['SideBDeaths'].astype('int64')

In [216]:
killed_exec_counts = coup.groupby('year')['Killed'].count()

In [217]:
killed_exec_war_death_counts = pd.merge(war_death_counts, killed_exec_counts, left_index=True, right_index=True)
killed_exec_war_death_counts['Year'] = killed_exec_war_death_counts.index
killed_exec_war_death_counts = killed_exec_war_death_counts[killed_exec_war_death_counts['Year'] > 1946]

# Viz 2

In [218]:
base = alt.Chart(killed_exec_war_death_counts).encode(alt.X('Year:O'))
line = base.mark_line(stroke='#e50000').encode(alt.Y('Total Deaths:Q', axis=alt.Axis(title='Total deaths from intra-state war', titleColor='#e50000')))
bar = base.mark_bar(stroke='#0343df').encode(alt.Y('Killed', axis=alt.Axis(title='Killed executives', titleColor='#0343df')))

In [219]:
viz2 = alt.layer(bar, line).resolve_scale(y = 'independent')

# Viz 3

In [220]:
intrastate_coup_concat

,cowcode,event_type1,event_type2,Year,Month,StateAbb,StateNme
0,483.0,Coup,Attempted coup,2006.0,4.0,CHA,Chad
1,950.0,Coup,Successful coup,2006.0,12.0,FIJ,Fiji
2,800.0,Coup,Successful coup,2006.0,9.0,THI,Thailand
3,840.0,Coup,Conspiracy,2006.0,2.0,PHI,Philippines
4,552.0,Coup,Conspiracy,2006.0,3.0,ZIM,Zimbabwe
...,...,...,...,...,...,...,...
1277,520.0,Intra-state war,Civil war for central control,2006.0,3.0,SOM,Somalia
1278,530.0,Intra-state war,Civil war for central control,2006.0,10.0,ETH,Ethiopia
1279,530.0,Intra-state war,Civil war for central control,2006.0,10.0,ETH,Ethiopia
1280,531.0,Intra-state war,Civil war for central control,2006.0,10.0,ERI,Eritrea


In [221]:
intrastate_coup_concat['Month'].replace(-9, np.nan, inplace=True)
intrastate_coup_concat.dropna(axis=0, how='any', inplace=True)

In [222]:
viz3 = alt.Chart(intrastate_coup_concat).mark_bar().encode(alt.X('Month:O'), alt.Y('count():Q', title='# of events'), alt.Column('event_type1', title='Coup / Intra-state War'), alt.Color('event_type2', title= 'Event Type'))

# Saving Specs

In [223]:
jekyll_dir = ('C:/Users/emory/OneDrive/Desktop/data viz/ebrigden.github.io/assets/json')
dash.save(jekyll_dir+'/final_viz_1.json')
viz2.save(jekyll_dir+'/final_viz_2.json')
viz3.save(jekyll_dir+'/final_viz_3.json')